# 👖 Autoencoders on Fashion MNIST

In [ ]:
working_dir = "/home/mary/work/repos/generative_deep_Learning_2nd_edition_pytorch"

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Add the path to the notebooks folder
notebooks_path = os.path.abspath(working_dir)
if notebooks_path not in sys.path:
    sys.path.append(notebooks_path)

In [ ]:
import torchvision
from torchvision import transforms
import torch
from notebooks.utils import display

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import math

## 0. Parameters <a name="parameters"></a>


In [ ]:
IMAGE_SIZE = 32
CHANNELS = 1
BATCH_SIZE = 100
BUFFER_SIZE = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 2
EPOCHS = 3

## 1. Prepare the data <a name="prepare"></a>

In [ ]:
data_dir = working_dir + "/data"

In [ ]:
import torch.utils
import torch.utils.data


transform = transforms.Compose([
    transforms.Pad((2, 2, 2, 2)), # Padding (left, top, right, bottom)
    transforms.ToTensor()
    
])

train_data = torchvision.datasets.FashionMNIST(data_dir, train=True, transform=transform, download=True)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_data = torchvision.datasets.FashionMNIST(data_dir, train=False, transform=transform, download=True)

test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

In [ ]:
print(f"training data size= {len(train_data)}")
print(f"test data size= {len(test_data)}")

In [ ]:
dataiter = iter(train_data_loader)
images, lables = next(dataiter)

print(type(images))
print(images.shape)

In [ ]:
display(images[:10])
print(lables[:10])

In [ ]:
class Encoder(nn.Module):

    def __init__(self, image_size, channels):
        super().__init__()

        p = self._get_padding_size(image_size, 2, 3)
        self.conv1 = nn.Conv2d(in_channels=channels, out_channels=32, kernel_size=3, stride=2, padding=p)
        
        p = self._get_padding_size(image_size/2, 2, 3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=p)

        p = self._get_padding_size(image_size/4, 2, 3)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=p)

        self.shape_before_flattening = (128, image_size/8, image_size/8)

        self.fc1 = nn.Linear(in_features=int(math.prod(self.shape_before_flattening)), out_features=EMBEDDING_DIM)

    def get_shape_before_flattening(self):
        return self.shape_before_flattening
    
    @staticmethod
    def _get_padding_size(input_w, stride, kernal_size):
        p = ((input_w /2) - 1) * stride
        p = (p - input_w) + kernal_size
        p = math.ceil(p/2)

        return p
    
    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        # # flatten
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        
        return x


In [ ]:
class Decoder(nn.Module):
    def __init__(self, shape_before_flatten):
        super().__init__()

        self.shape_before_flatten = shape_before_flatten

        self.fc1 = nn.Linear(in_features=EMBEDDING_DIM, out_features=int(math.prod(self.shape_before_flatten)))

        p = self._get_padding_size(self.shape_before_flatten[1], stride=2, kernaal_size=3)
        self.conv_trans1 = nn.ConvTranspose2d(in_channels=self.shape_before_flatten[0], out_channels=128, 
                                              kernel_size=3, stride=2, padding=1, output_padding=1)
        
        p = self._get_padding_size(self.shape_before_flatten[1]*2, stride=2, kernaal_size=3)
        self.conv_trans2 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, 
                                              stride=2, padding=p, output_padding=1)
        
        p = self._get_padding_size(self.shape_before_flatten[1]*4, stride=2, kernaal_size=3)
        self.conv_trans3 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3,
                                              stride=2, padding=p, output_padding=1)
        
        self.conv1 = nn.Conv2d(in_channels=32, out_channels=CHANNELS, kernel_size=3,
                               stride=1, padding='same')
        
    @staticmethod
    def _get_padding_size(input_w, stride, kernaal_size):
        p = ((input_w - 1) * stride) / 2
        p = p - input_w
        p = p + (kernaal_size / 2)
        p = p + 1/2
        return math.ceil(p)
    
    def forward(self, x):
        x = self.fc1(x)
        c, w, h = self.shape_before_flatten
        x = x.view(-1, c, w, h)
        x = self.conv_trans1(x)
        x = F.relu(x)
        x = self.conv_trans2(x)
        x = F.relu(x)
        x = self.conv_trans3(x)
        x = F.relu(x)
        x = self.conv1(x)
        x = F.sigmoid(x)
        # Should we add sigmoid?

        return x
        

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, image_size, channels):
        super().__init__()

        self.encoder = Encoder(image_size=image_size, channels=channels)
        self.shape_before_flatten = tuple(map(int, self.encoder.get_shape_before_flattening()))
        self.decoder = Decoder(self.shape_before_flatten)
    
    def forward(self, x):
        emb = self.encoder(x)
        img = self.decoder(emb)

        return img

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = Encoder(32, 1).to(device)
print(encoder)

In [ ]:
summary(encoder, (1, 32, 32))

In [ ]:
shape_before_flatten = encoder.get_shape_before_flattening()
shape_before_flatten = tuple(map(int, shape_before_flatten))

decoder = Decoder(shape_before_flatten).to(device)
print(decoder)

In [ ]:
summary(decoder, (2,))

In [ ]:
auto_encoder = AutoEncoder(32, 1).to(device)
print(auto_encoder)

In [ ]:
summary(auto_encoder, (1, 32, 32))

## 3. Train the autoencoder <a name="train"></a>

In [ ]:
num_epoch = 5
learning_rate = 0.0005

In [ ]:
# loss_fn = nn.BCEWithLogitsLoss()
loss_fn = nn.BCELoss()

In [ ]:
optmizer = optim.Adam(auto_encoder.parameters(), lr=learning_rate)

In [ ]:
def fit(model, train_dataloader, optimizer, loss_fn, epochs=10):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # loop over the number of epoch
    for i in range(epochs):
        #  set the model for training
        model.train()
        # loop over the dataloader to get all the data
        running_loss = 0.0
        num_samples = 0
        correct = 0
        for images, _ in train_dataloader:
            #  zero the gradiants of the optimizer
            optimizer.zero_grad()
            # move the training data to the same device as the model
            images = images.to(device)
            # Predict the lables
            predictions = model(images)
            # calculate the loss
            loss = loss_fn(predictions, images)
            # calcualte the gradients for the loss
            loss.backward()
            # updat the weights using the optimizer
            optimizer.step()
            # accumilate the loss
            running_loss += loss.item()

            # calcualte the accuracy
            _,pred_lable = torch.max(predictions, 1)
            # _, corr_label = torch.max(labels, 1)
        
        print( f"Epoch {i} / {epochs}: loss= {running_loss/len(train_dataloader):.4f}")

            

In [ ]:
fit(auto_encoder, train_data_loader, optmizer, loss_fn, num_epoch)

In [ ]:
test_iter = iter(test_data_loader)
test_images, test_lables = next(test_iter)

In [ ]:
emb = auto_encoder.encoder(test_images.to(device))
gen_images = auto_encoder.decoder(emb)

In [ ]:
display(test_images[:10])
display(gen_images[:10])

In [ ]:
emb_train = auto_encoder.encoder(images.to(device))
gen_images_train = auto_encoder.decoder(emb_train)

In [ ]:
display(images[:10])
display(gen_images_train[:10])